# Summerize Shareholding:

#### Summerize: 

- For all stocks from RawReports folder, read specific Excels sheet into dataframe  
- Add stock name column to df 
- Transform the df 
- Consolidate the df 

#### Analyze:

- Compute something
- Write the computed new data 
- Analyze or visualize the new data 

#### Summary Report File Name Convention: 
- Summary01-Top-Ratios 
- Summary02-Quarterly-Results
- Summary03-Profit-And-Loss
- Summary04-Compounded-Sales-Growth
- Summary05-Compounded-Profit-Growth
- Summary06-Stock-Price-CAGR 
- Summary07-Return-on-Equity
- Summary08-Balance-Sheet
- Summary09-Cash-Flows
- Summary10-Ratios
- Summary11-Shareholding-Pattern

## 1.1 Imports:

In [1]:
import requests as rq
import pandas as pd 
import time
import datetime
from bs4 import BeautifulSoup
import re
import os


## 1.2 Define Inputs:

>  ⚠️ <span style="color:red"> **Update the CSV File below to read as needed:**</span>

In [2]:
## CAREFULLY UPDATE THIS. This should be same as what specified in RawReport notebook

# For My Watchlisted stocks
input_cvs_file = "../Input/" + "stocks-watchlisted.csv"
# For PSU stocks
input_cvs_file = "../Input/" + "stocks-psu.csv"

input_cvs_file = "../Input/" + "stocks-watchlisted.csv"

## UPDATE THIS. The folder is in  ../Output/RawReports/. This is where raw reports are read and summerized

raw_report_folder_name     = "20240817-132127"


In [3]:
raw_report_folder_path = "../Output/RawReports/" + raw_report_folder_name

df = pd.read_csv(input_cvs_file)
df.head(5)


,Screener Stock Symbol,Url Segment
0,526433,consolidated
1,532407,consolidated
2,544021,consolidated
3,AARTIDRUGS,consolidated
4,AARTIIND,consolidated


## 1.3 Process reports:

In [4]:
df_shareholding_pattern = pd.DataFrame([])

for index, row in df.iterrows():
    stock_symbol = row["Screener Stock Symbol"]
    filepath = raw_report_folder_path + "/" + stock_symbol + ".xlsx"
    df_data =  pd.read_excel(filepath, sheet_name="Shareholding Pattern")
    print("Processing stock ",stock_symbol)
    # Add stockname column and move into 2nd position: 
    df_data['Stockname'] = stock_symbol
    column_to_move = df_data.pop("Stockname")
    df_data.insert(1, "Stockname", column_to_move)
    df_shareholding_pattern = pd.concat([df_shareholding_pattern, df_data], ignore_index=True, sort=False)
    
print("All done!")    

Processing stock  526433
Processing stock  532407
Processing stock  544021
Processing stock  AARTIDRUGS
Processing stock  AARTIIND
Processing stock  AARTIPHARM
Processing stock  AARTISURF
Processing stock  ABB
Processing stock  ACC
Processing stock  ADANIENT
Processing stock  AFFLE
Processing stock  ALKYLAMINE
Processing stock  ANGELONE
Processing stock  ARE&M
Processing stock  ASAHIINDIA
Processing stock  ASHOKLEY
Processing stock  ASIANPAINT
Processing stock  ASTRAL
Processing stock  AXISBANK
Processing stock  BAJAJ-AUTO
Processing stock  BAJAJELEC
Processing stock  BAJAJFINSV
Processing stock  BAJFINANCE
Processing stock  BALAMINES
Processing stock  BATAINDIA
Processing stock  BEL
Processing stock  BERGEPAINT
Processing stock  BHARTIARTL
Processing stock  BOROLTD
Processing stock  BRITANNIA
Processing stock  BSE
Processing stock  CAMS
Processing stock  CAMPUS
Processing stock  CDSL
Processing stock  CENTUM
Processing stock  CGPOWER
Processing stock  COCHINSHIP
Processing stock  COFO

## 1.5 Inspect dataframe:

In [5]:
print(len(df_shareholding_pattern))

879


In [6]:
df_shareholding_pattern.describe()

,Shareholding Pattern,Stockname,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Jul 2024,Sep 2021,Jun 2021
count,879,879,808,813,823,823,828,834,839,860,879,874,879,16,777,5
unique,7,160,720,728,733,735,734,743,750,766,782,786,790,16,689,5
top,FIIs,JUBLFOOD,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,57.35%,0.00%,20.00%
freq,160,7,42,38,35,12,12,10,9,11,11,10,9,1,41,1


In [7]:
df_shareholding_pattern.columns

Index(['Shareholding Pattern', 'Stockname', 'Dec 2021', 'Mar 2022', 'Jun 2022',
       'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023', 'Sep 2023', 'Dec 2023',
       'Mar 2024', 'Jun 2024', 'Jul 2024', 'Sep 2021', 'Jun 2021'],
      dtype='object')

## 1.6 Sort column ordering:

In [8]:
date = '1970-01-01'
df_shareholding_pattern = df_shareholding_pattern.sort_index(
    axis=1, 
    key=lambda x: pd.to_datetime(x, errors='coerce', format='mixed').fillna(date)
)

In [9]:
df_shareholding_pattern.columns

Index(['Shareholding Pattern', 'Stockname', 'Jun 2021', 'Sep 2021', 'Dec 2021',
       'Mar 2022', 'Jun 2022', 'Sep 2022', 'Dec 2022', 'Mar 2023', 'Jun 2023',
       'Sep 2023', 'Dec 2023', 'Mar 2024', 'Jun 2024', 'Jul 2024'],
      dtype='object')

In [10]:
df_shareholding_pattern.head(5)

,Shareholding Pattern,Stockname,Jun 2021,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Jul 2024
0,Promoters,526433,NaN,NaN,61.84%,61.84%,62.51%,62.50%,62.50%,62.50%,62.79%,62.79%,62.79%,61.54%,61.54%,57.35%
1,FIIs,526433,NaN,NaN,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.02%,0.00%,0.66%,0.25%,0.23%
2,DIIs,526433,NaN,NaN,0.02%,0.02%,0.02%,0.02%,0.02%,0.02%,0.02%,0.00%,0.02%,0.00%,0.00%,0.00%
3,Public,526433,NaN,NaN,38.14%,38.14%,37.47%,37.48%,37.48%,37.48%,37.19%,37.19%,37.19%,37.81%,38.21%,42.41%
4,No. of Shareholders,526433,NaN,NaN,8754,15469,16612,17272,16987,16588,16574,17103,17219,17460,18746,18752


## 1.7 Save summary report:

In [11]:
output_path = "../Output/SummaryReports/"+"Summary11-Shareholding-Pattern-"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+".csv"

df_shareholding_pattern.to_csv(output_path,index=False,sep=",")